In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.preprocessing import StandardScaler
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# #df=pd.read_csv('cleaned2_df.csv',index_col=0)
# df=pd.read_csv('cleaned_df.csv',index_col=0)


In [3]:
# df_sample=df.sample(50000,random_state=42)
# X=df_sample.drop('smoking',axis=1)
# y=df_sample['smoking']


In [4]:
df=pd.read_csv('balanced_cleaned_train_dataset.csv')


In [5]:
test_set=pd.read_csv('testset.csv')

X=df.drop('smoking',axis=1)
y=df['smoking']


X_test= test_set.drop('smoking', axis=1)
y_test = test_set['smoking']

In [6]:
# duplicates = pd.merge(df, test_set, how='inner')
# duplicates.head()

In [ ]:

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 100, 300, step=20)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)

    scores = []

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        
        #scaler = MinMaxScaler()
        # scaler =StandardScaler()
        # X_train_scaled = scaler.fit_transform(X_train)
        # X_val_scaled = scaler.transform(X_val)

        
        model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_jobs=-1,
            random_state=42
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        scores.append(acc)

    return np.mean(scores)

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize',sampler=sampler)
study.optimize(objective, n_trials=20)  


print("best_params:", study.best_params)
print(f"best_value: {study.best_value:.4f}")

[I 2025-08-14 11:32:40,928] A new study created in memory with name: no-name-a0de816f-d324-4897-be4e-d67de85378ea
[I 2025-08-14 11:32:42,400] Trial 0 finished with value: 0.8205 and parameters: {'n_estimators': 180, 'max_depth': 10, 'learning_rate': 0.1205712628744377}. Best is trial 0 with value: 0.8205.
[I 2025-08-14 11:32:42,946] Trial 1 finished with value: 0.8296000000000001 and parameters: {'n_estimators': 220, 'max_depth': 4, 'learning_rate': 0.01699897838270077}. Best is trial 1 with value: 0.8296000000000001.
[I 2025-08-14 11:32:43,748] Trial 2 finished with value: 0.8227500000000001 and parameters: {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.07725378389307355}. Best is trial 1 with value: 0.8296000000000001.
[I 2025-08-14 11:32:44,155] Trial 3 finished with value: 0.8231999999999999 and parameters: {'n_estimators': 240, 'max_depth': 3, 'learning_rate': 0.27081608642499677}. Best is trial 1 with value: 0.8296000000000001.
[I 2025-08-14 11:32:44,824] Trial 4 finish

best_params: {'n_estimators': 280, 'max_depth': 4, 'learning_rate': 0.01855998084649058}
best_value: 0.8298


In [8]:
model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=4,
        learning_rate=0.01855998084649058,
        n_estimators=280,
        eval_metric='logloss',
        random_state=42
    )
model.fit(X, y)  

y_test_pred = model.predict(X_test)

acc_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='macro')
recall_test = recall_score(y_test, y_test_pred, average='macro')
f1_test = f1_score(y_test, y_test_pred, average='macro')

train_pred = model.predict(X)  
train_acc = accuracy_score(y, train_pred)
print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

print(f"Test set Accuracy:  {acc_test:.4f}")
print(f"Test set Precision: {precision_test:.4f}")
print(f"Test set Recall:    {recall_test:.4f}")
print(f"Test set F1 Score:  {f1_test:.4f}")
print("\nTest set classification_report:")
print(classification_report(y_test, y_test_pred, digits=4))

Train Accuracy: 0.8343, test Accuracy: 0.8380
Test set Accuracy:  0.8380
Test set Precision: 0.8488
Test set Recall:    0.8380
Test set F1 Score:  0.8367

Test set classification_report:
              precision    recall  f1-score   support

           0     0.9102    0.7500    0.8224      1000
           1     0.7874    0.9260    0.8511      1000

    accuracy                         0.8380      2000
   macro avg     0.8488    0.8380    0.8367      2000
weighted avg     0.8488    0.8380    0.8367      2000



In [ ]:

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    
    #scaler = MinMaxScaler()
    # scaler =StandardScaler()
    # X_train_scaled = scaler.fit_transform(X_train)
    # X_val_scaled = scaler.transform(X_val)

    
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=4,
        learning_rate=0.01855998084649058,
        n_estimators=280,
        eval_metric='logloss',
        random_state=42
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred)

    y_train_pred=model.predict(X_train)
    train_acc= accuracy_score(y_train,y_train_pred)

    print(f"Train Accuracy: {train_acc:.4f}, val Accuracy: {acc:.4f}")







    



Train Accuracy: 0.8353, val Accuracy: 0.8270
Train Accuracy: 0.8339, val Accuracy: 0.8340
Train Accuracy: 0.8348, val Accuracy: 0.8303
Train Accuracy: 0.8339, val Accuracy: 0.8310
Train Accuracy: 0.8356, val Accuracy: 0.8267
